In [42]:
import pandas as pd
import numpy as np
import requests
import json

In [44]:
target = 'opened' # 'opened', 'closed'
dict_target = {
    'opened': '액티브',
    'closed': '휴폐업',
}
kor_target = dict_target[target]

In [45]:
df_closed_company = pd.read_csv(f'data/{target}_company_summary_valid.csv', index_col=0)

In [46]:
df_closed_company.head()

,BIZ_NO,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,...,NATN_NM,EMP_CNT,IS_LP,IS_LC,IS_LTD,IS_SP,IS_ETC,HAS_HOMEPAGE,CEO_CNT,label
0,2218116923,M,2,2,1,1,N,N,NaN,10.0,...,NaN,12.0,False,False,True,False,False,False,2.0,0
1,1138111739,M,2,2,1,1,N,N,NaN,10.0,...,NaN,43.0,False,False,True,False,False,True,2.0,0
2,1108146490,M,2,2,1,1,N,N,NaN,10.0,...,NaN,3.0,False,False,True,False,False,True,1.0,0
3,3128134737,M,2,2,1,1,N,N,NaN,10.0,...,NaN,24.0,False,False,True,False,False,True,1.0,0
4,2078132193,M,2,2,1,1,N,N,NaN,10.0,...,NaN,35.0,False,False,True,False,False,True,2.0,0


In [47]:
descs = df_closed_company.MN_BIZ_CONT

In [48]:
biz_nos = df_closed_company.BIZ_NO

In [49]:
data_all_closed_biz = pd.read_excel(f'data/{kor_target} 중소법인 재무보유.xlsx', sheet_name='기업개요')

In [50]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
#  Load Model and Tokenize
tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/kobart-news")
model = BartForConditionalGeneration.from_pretrained("ainize/kobart-news")
# Encode Input Text
input_text = '국내 전반적인 경기침체로 상가 건물주의 수익도 전국적인 감소세를 보이고 있는 것으로 나타났다. 수익형 부동산 연구개발기업 상가정보연구소는 한국감정원 통계를 분석한 결과 전국 중대형 상가 순영업소득(부동산에서 발생하는 임대수입, 기타수입에서 제반 경비를 공제한 순소득)이 1분기 ㎡당 3만4200원에서 3분기 2만5800원으로 감소했다고 17일 밝혔다. 수도권, 세종시, 지방광역시에서 순영업소득이 가장 많이 감소한 지역은 3분기 1만3100원을 기록한 울산으로, 1분기 1만9100원 대비 31.4% 감소했다. 이어 대구(-27.7%), 서울(-26.9%), 광주(-24.9%), 부산(-23.5%), 세종(-23.4%), 대전(-21%), 경기(-19.2%), 인천(-18.5%) 순으로 감소했다. 지방 도시의 경우도 비슷했다. 경남의 3분기 순영업소득은 1만2800원으로 1분기 1만7400원 대비 26.4% 감소했으며 제주(-25.1%), 경북(-24.1%), 충남(-20.9%), 강원(-20.9%), 전남(-20.1%), 전북(-17%), 충북(-15.3%) 등도 감소세를 보였다. 조현택 상가정보연구소 연구원은 "올해 내수 경기의 침체된 분위기가 유지되며 상가, 오피스 등을 비롯한 수익형 부동산 시장의 분위기도 경직된 모습을 보였고 오피스텔, 지식산업센터 등의 수익형 부동산 공급도 증가해 공실의 위험도 늘었다"며 "실제 올 3분기 전국 중대형 상가 공실률은 11.5%를 기록하며 1분기 11.3% 대비 0.2% 포인트 증가했다"고 말했다. 그는 "최근 소셜커머스(SNS를 통한 전자상거래), 음식 배달 중개 애플리케이션, 중고 물품 거래 애플리케이션 등의 사용 증가로 오프라인 매장에 영향을 미쳤다"며 "향후 지역, 콘텐츠에 따른 상권 양극화 현상은 심화될 것으로 보인다"고 덧붙였다.'
input_ids = tokenizer.encode(input_text, return_tensors="pt")
# Generate Summary Text Ids
summary_text_ids = model.generate(
    input_ids=input_ids,
    bos_token_id=model.config.bos_token_id,
    eos_token_id=model.config.eos_token_id,
    length_penalty=2.0,
    max_length=142,
    min_length=56,
    num_beams=4,
)
# Decoding Text
print(tokenizer.decode(summary_text_ids[0], skip_special_tokens=True))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


전반적인 경기 침체로 건물주 수익이 전국적인 감소세를 보이면서 전국 중대형 상가 순영업소득이 3분기 2만5800원으로 감소했으며 조현택 상가정보연구소 연구원은 앞으로 지역, 콘텐츠에 따른 상권 양극화 현상이 심화될 것으로 보인다고 밝혔다.


In [51]:
def kor_summarize(input_text, max_length=120, min_length=20, length_penalty=2.0, num_beams=5):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=length_penalty,
        max_length=max_length,
        min_length=min_length,
        num_beams=num_beams,
    )
    return tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

In [52]:
sumarized_descs = []

In [53]:
num_alpha = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
list_num_alpha = list(num_alpha)

In [54]:
for desc in descs:
    try:
        if np.isnan(desc):
            sumarized_descs.append('')
            break
            continue
    except:
        response = kor_summarize(
            input_text=desc,
            max_length=8,
            min_length=3,
            length_penalty=0,
            num_beams=12
        )
        if len(response) < 2:
            continue
        elif len(response) > 7:
            continue
        elif any(char in list_num_alpha for char in response):
            continue
        sumarized_descs.append(response)
        
        

In [39]:
(sumarized_descs)


['자동차',
 '자동차드업',
 '일기',
 '전자제휴용기',
 '침구류류와',
 '다공청소시스템',
 '지난판  봉 ',
 '이약',
 '실내건축, 모',
 '금자동조리기',
 '목재재생산업체',
 '\n물제조업업',
 '유류 및류 및',
 '국내',
 '백백백 외 외',
 '임대업',
 '용용접봉,',
 '수정수배전',
 '하수수처리업',
 '전착도장설비',
 '은나노 분말',
 '조조경공공',
 '의류, 원단',
 '\n유피혁혁',
 '보링그라우팅',
 '지류 도,소',
 '지덕판금공',
 '전선용용용빅',
 '공기조화기',
 '살균제, 살균',
 '토목의 목 목',
 '책업업업업',
 '통신기부품',
 '자동차운송업업',
 '공공사업공',
 '소소음방지시설',
 '토목공사업에서',
 '기계',
 '수출',
 '수산물 도매업',
 '저저항기기',
 '토공사업사업토',
 '화장',
 '습식방수공사',
 '\n업',
 '전기',
 '조조경공공',
 '토목건설업업',
 '구구제조업으로',
 '해기업의기업의',
 '엘월',
 '지난',
 '재재활용 재생',
 '창창고자동',
 '책업업업업',
 '주업.',
 '선식식은선선',
 "'용용용 기계",
 '지난대',
 '프로그램업업',
 '피대코팅 제조',
 '관관제조업업',
 '도판, 철판',
 '수업업',
 '염색가공업,',
 '제체 및 엘씨',
 '이이형필',
 '의의류류류',
 '대업업업업',
 '우레탄, 에',
 '베니어합판,',
 '가정롱, 서랍',
 '단호박, 단',
 '인쇄 출판업',
 '소산물과 건어',
 '객이요',
 '소프트',
 '철봉.철봉',
 '전기',
 '인삼식품이',
 '전기',
 '\n동물 훈제 ',
 '프프레이밸브',
 '통신',
 '스텐텐레스 프',
 '포포장용기',
 '목목설계용역',
 '도소매업업',
 '\n역',
 '체육',
 '냉동기계진열',
 '방화셔터,',
 '수산물 도매업',
 '고고보관업',
 '최근.규조토',
 '판매업 판매업',
 '판매업 판매업',
 '특수고무에'

In [40]:
sumarized_descs1 = []
for desc in sumarized_descs:
    desc1 = desc.replace('\n', '')
    sumarized_descs1.append(desc1)

In [41]:
with open('text_keywords1.txt', 'w') as f:
    f.write('\n'.join(sumarized_descs1))
